In [1]:
# Initial imports
import numpy as np
import pandas as pd


import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
from pathlib import Path

# %matplotlib inline

#Load .env environment variables
load_dotenv()

faangm_closing_return_df_=pd.read_csv(Path("AMZN_min.csv"))
faangm_closing_return_df_.index = faangm_closing_return_df_["timestamp"]
faangm_closing_return_df_.drop(columns="timestamp", inplace=True)
faangm_closing_return_df_.head()

,AMZN
timestamp,
2021-09-29 08:06:00+00:00,3350.0
2021-09-29 08:13:00+00:00,3345.0
2021-09-29 08:15:00+00:00,3347.0
2021-09-29 08:49:00+00:00,3342.0
2021-09-29 09:07:00+00:00,3350.0


In [ ]:
# Initialze Lists to be used for creation of Datafreame having Input Parameters, and output Algo Trading Backtesting results
# Lists for Input Params


# Lists for Backtesting results
annual_return_list=[]
cumulative_returns_list=[]
annual_volatility_list=[]
sharpe_ratio_list=[]

periods_list=[]
oversold_list=[]
overbought_list=[]

# 'For' loops that use each intended combination of Parameters to run the full Algo Trading Model and Backtesting 
# and append the results in the Lists initialized above
for periods in range(5,30): # number of periods to calculate the %K line of the Stochastic Oscillator 
    for oversold in range(15,35): # number of periods to calculate the %D line of the Stochastic Oscillator 
        for overbought in range(60,90): # Short Window to calculate the Simple Moving Average Fast Indicator
            faangm_closing_return_df_rsi = faangm_closing_return_df_.copy()
            
            
            close_delta = df['AMZN'].diff()

            # Make two series: one for lower closes and one for higher closes
            up = close_delta.clip(lower=0)
            down = -1 * close_delta.clip(upper=0)
            
            ema = True

            if ema == True:
                # Use exponential moving average
                ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
                ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
            else:
                # Use simple moving average
                ma_up = up.rolling(window = periods, adjust=False).mean()
                ma_down = down.rolling(window = periods, adjust=False).mean()

            rsi = ma_up / ma_down
            rsi = 100 - (100/(1 + rsi))
            
            faangm_closing_return_df_rsi['RSI']= rsi
            
            
            faangm_closing_return_df_rsi.dropna(inplace=True)

            df_1=faangm_closing_return_df_rsi.copy()
            
            prices= df_1["AMZN"].array
            rsi= df_1["RSI"].array
            
            
            rsi_signal = []
            signal = 0

            for i in range(len(prices)):
                if (rsi[i] < oversold) :
                    if signal != 1:
                        signal = 1                
                        rsi_signal.append(signal)
                    else:
                        rsi_signal.append(0)
                elif (rsi[i] > overbought):
                    if (signal == 1):
                        signal = -1                
                        rsi_signal.append(signal)
                    else:
                        rsi_signal.append(0)
                else:
                    rsi_signal.append(0)
                    
            df_1["AMZN_rsi_entry_exit"]= rsi_signal
            df_1["AMZN_rsi_balance"]= df_1["AMZN_rsi_entry_exit"].cumsum()
            df_1 = df_1.dropna()
            
            initial_capital = float(1000000)
            # Set the share size
            share_size = 500

            # Take a 500 share position where the dual moving average crossover is 1 (sma_fast is greater than sma_slow)
            df_1['AMZN_position'] = share_size * df_1['AMZN_rsi_entry_exit']

            # Find the points in time where a 500 share position is bought or sold
            df_1['AMZN_entry_exit_position'] = df_1['AMZN_position'].diff()

            # Multiply share price by entry/exit positions and get the cumulatively sum
            df_1['AMZN_portfolio_holdings'] = df_1['AMZN'] * df_1['AMZN_entry_exit_position'].cumsum()

            # Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
            df_1['AMZN_portfolio_cash'] = initial_capital - (df_1['AMZN'] * df_1['AMZN_entry_exit_position']).cumsum()

            # Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
            df_1['AMZN_portfolio_total'] = df_1['AMZN_portfolio_cash'] + df_1['AMZN_portfolio_holdings']

            # Calculate the portfolio daily returns
            df_1['AMZN_portfolio_daily_returns'] = df_1['AMZN_portfolio_total'].pct_change()

            # Calculate the cumulative returns
            df_1['AMZN_portfolio_cumulative_returns'] = (1 + df_1['AMZN_portfolio_daily_returns']).cumprod() - 1

            # Drop all NaN values from the DataFrame
            df_1 = df_1.dropna()

            
            periods_list.append(periods)
            oversold_list.append(periods)
            overbought_list.append(periods)


            # Calculate cumulative return
            cumulative_returns_list.append(df_1['AMZN_portfolio_cumulative_returns'][-1])

            # Calculate annualized return
            annual_return_list.append( (df_1['AMZN_portfolio_daily_returns'].mean() * 252 * 8 * 60) )

            # Calculate annual volatility
            annual_volatility_list.append( ( df_1['AMZN_portfolio_daily_returns'].std() * np.sqrt(252 * 8 * 60) ))

            # Calculate Sharpe Ratio
            sharpe_ratio_list.append( ( df_1['AMZN_portfolio_daily_returns'].mean() * 252 * 8 * 60) / ( faangm_closing_return_df['AMZN_portfolio_daily_returns'].std() * np.sqrt(252 * 8 * 60)))


            
# Prepare DataFrame for Backtesting Results metrics evaluation for each Parameter combination
AMZN_portfolio_evaluation_df = pd.DataFrame({'periods': periods_list ,
                                             'overbought': oversold_list ,
                                             'oversold': overbought_list ,
                                             'annual_return': annual_return_list ,
                                             'cumulative_returns': cumulative_returns_list ,
                                             'annual_volatility': annual_volatility_list,
                                             'sharpe_ratio': sharpe_ratio_list })

# Write the output in a file
AMZN_portfolio_evaluation_df.to_csv("param_sweep_rsi.csv")

AMZN_portfolio_evaluation_df.head()